# RAG com LCEL

Agora vamos para exemplos mais avançados de chains e como podemos reproduzí-las utilizando LCEL.

Vamos começar mostrando como fazer o processos de RAG. Para isso, o processo de criação da vector store é exatamente igual ao que vimos no curso de Aplicações com IA com LangChain.

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

CRIANDO CHUNKS - Carregando e particionando o documento

In [5]:
PATHS = [
    "arquivos\Explorando o Universo das IAs com Hugging Face.pdf"
]

pages = []

for path in PATHS:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

In [12]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap = 50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(pages)

EMBEDDINGS - Importando e instanciando o OpenAiEmbeddings

In [9]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

C:\Users\edinocencio\AppData\Local\Temp\ipykernel_8548\22710313.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


STORAGE - Amrazenando os dados em um VECTOR STORE

In [15]:
from langchain_community.vectorstores.faiss import FAISS

vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,

)

retriever = vector_store.as_retriever(search_type="mmr")


Agora na criação da chain é que temos uma mudança. Podemos definir nossa chain manualmente da seguinte forma

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import  StrOutputParser
from langchain_openai import ChatOpenAI
from regex import template

llm = ChatOpenAI()

template = """ Reponda as seguintes perguntas ao usuário utilizando apenas o contexto fornecido.

Contexto: {context}

Pergunta: {pergunta}

"""
template = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
from certifi import contents
from langchain_core.runnables import  RunnablePassthrough, RunnableParallel

# RunnableParallel: Provavelmente é uma classe que permite executar múltiplos componentes em paralelo.
# 'pergunta': Um componente que utiliza RunnablePassthrough, que parece ser uma classe que simplesmente passa os dados sem modificações.
# 'context': Um componente que utiliza retriever, que deve ser um objeto ou função responsável por recuperar algum tipo de dados ou contexto.

setup_and_retrievel = RunnableParallel(
    {
        'pergunta': RunnablePassthrough(),
        'context': retriever
     }
)

